In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems
from src.embeddings import RecsysEmbedding

experiment_name = 'SCOT'
device = 'cuda:0'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)


class SCOT(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1, debug=False):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.nheads = nheads
        self.debug = debug
        self.attention= torch.nn.MultiheadAttention(
            self.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.out_layer = torch.nn.Sequential(
                # torch.nn.LayerNorm(2* embedding.embedding_dim),
                # torch.nn.Linear(embedding.embedding_dim * 2, embedding.embedding_dim * 2),
                # torch.nn.GELU(),
                torch.nn.Linear(embedding.embedding_dim * 2, output_dim)
        )

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        device = item_embs.device
        
        if self.debug: print('responses', batch['responses'])
        
        # flattening slates into long sequences
        item_embs = item_embs.flatten(1, 2)
        
        # getting user embedding (mean consumed items)
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2)
        
        # adding a user embedding as a 'zero item' to 
        # make predictions if nothing is observed
        keys = torch.cat([
                user_embs[:, None, :],
                item_embs
            ],
            dim = 1
        )
        
        clicked_mask = batch['responses'].flatten(1, 2) > 0
        if self.debug: print(clicked_mask, clicked_mask.shape)
        clicked_mask = torch.cat([
                torch.ones_like(clicked_mask[:, 0, None]).bool(),
                clicked_mask
            ],
            dim=-1
        )
        if self.debug: print('clicked_mask:', clicked_mask, clicked_mask.shape)
        
        clicked_items = [
            keys[i][clicked_mask[i], :]
            for i in range(shp[0])
        ]
        keys = torch.nn.utils.rnn.pad_sequence(
            clicked_items,
            batch_first=True,
            padding_value=float('nan')
        )
        key_padding_mask = keys.isnan().all(-1)
        keys = keys.nan_to_num(0)
        
        if self.debug: print('key', keys.shape, keys, key_padding_mask)
        
        # forbid model looking into future (and into current iteraction)
        # at the end, mask will be (num_heads * bsize, slate_size * sequence_size, max_len_clicked_items)
        future_mask = torch.ones((item_embs.size(-2) + 1, item_embs.size(-2) + 1 ), device=device)
        future_mask = torch.triu(future_mask, diagonal=0)[1:, :].bool()
        if self.debug: print('future_mask', future_mask.shape, future_mask)
        ####### TODOTODOTODO ########
        # change future masl to be slatewise, not sequencewise
        # build it without large matrix generaton if possible
        # chunk previous iteracrtion
        ############################
        
        if self.debug: print('click_mask_repeated', clicked_mask[None, 0, ...].repeat(item_embs.size(-2), 1))
        
        if self.debug: print(
            'first user click & past', 
            future_mask[clicked_mask[None, 0, ...].repeat(item_embs.size(-2), 1)].reshape(item_embs.size(-2), keys.size(-2) )
        )
        
        attn_mask = [
            future_mask[
                clicked_mask[None, i, ...].repeat(item_embs.size(-2), 1)
            ].reshape(item_embs.size(-2), clicked_mask[None, i, ...].sum() ).T
            for i in range(shp[0])
        ]
        if self.debug: print('attn_mask', attn_mask[0].shape, attn_mask[1].shape )
                
        
        attn_mask = torch.nn.utils.rnn.pad_sequence(
            attn_mask,
            batch_first=True,
            padding_value=True
        ).permute(0, 2, 1)
        if self.debug: print('attn_mask_stacked', attn_mask)
        
        if self.debug: print(item_embs, keys, key_padding_mask, attn_mask.repeat_interleave(self.nheads, 0))
        features, attn_map = self.attention(
            item_embs, keys, keys,
            key_padding_mask=key_padding_mask,
            attn_mask = attn_mask.repeat_interleave(self.nheads, 0)
        )
        
        if self.debug: print(shp, item_embs.shape)
        features = torch.cat(
            [
                features.reshape(shp),
                item_embs.reshape(shp)
            ],
            dim = -1
        )
        
        return self.out_layer(features).squeeze(-1)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SCOT(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3142.59it/s]


Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.6666666269302368, 'accuracy': 0.25}


train... loss:0.7044374346733093:   0%|                                                                                                    | 1/5000 [00:00<28:57,  2.88it/s]

Val update: epoch: 0 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 0.6666666269302368 | treshold: 0.55
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 0.6666666269302368 | 


train... loss:0.6942538022994995:   0%|                                                                                                    | 2/5000 [00:00<28:51,  2.89it/s]

Val update: epoch: 1 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.55
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 


train... loss:0.6847817301750183:   0%|                                                                                                    | 3/5000 [00:01<35:38,  2.34it/s]

Val update: epoch: 3 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.5800000000000001
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SCOT(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (attention): MultiheadAttention(
     (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
   )
   (out_layer): Sequential(
     (0): Linear(in_features=64, out_features=1, bias=True)
   )
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SCOT(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating SCOT with svd embeddings
Test before learning: {'f1': 0.13685372471809387, 'roc-auc': 0.5017930269241333, 'accuracy': 0.8512182831764221}


train... loss:49.74499770998955:   0%|                                                                                                  | 1/5000 [01:01<85:40:05, 61.69s/it]

Val update: epoch: 0 |accuracy: 0.09982575476169586 | f1: 0.18153013288974762 | auc: 0.5062177181243896 | treshold: 0.01
Test: accuracy: 0.09984634816646576 | f1: 0.18156416714191437 | auc: 0.5073295831680298 | 


train... loss:34.518369287252426:   0%|                                                                                                 | 2/5000 [01:30<59:06:30, 42.58s/it]

Val update: epoch: 1 |accuracy: 0.12159887701272964 | f1: 0.18361753225326538 | auc: 0.5580906867980957 | treshold: 0.04
Test: accuracy: 0.12342137843370438 | f1: 0.18401013314723969 | auc: 0.5660172700881958 | 


...

train... loss:30.707155615091324:   0%|▏                                                                                              | 11/5000 [14:22<108:25:22, 78.24s/it]

Val update: epoch: 10 |accuracy: 0.8753183484077454 | f1: 0.2550275921821594 | auc: 0.7114785313606262 | treshold: 0.15000000000000002
Test: accuracy: 0.8766371607780457 | f1: 0.2708873748779297 | auc: 0.71410071849823 | 


train... loss:30.077650368213654:   0%|▎                                                                                              | 15/5000 [19:30<107:09:55, 77.39s/it]

Val update: epoch: 14 |accuracy: 0.8111903667449951 | f1: 0.3022564649581909 | auc: 0.7160630226135254 | treshold: 0.16
Test: accuracy: 0.8126143217086792 | f1: 0.3064507246017456 | auc: 0.7150951027870178 | 


train... loss:27.733672067523003:   1%|▊                                                                                              | 42/5000 [55:04<108:21:39, 78.68s/it]


In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [ ]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SCOT(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating SCOT with neural embeddings
Test before learning: {'f1': 0.6153877973556519, 'roc-auc': 0.5364928245544434, 'accuracy': 0.5471889972686768}


train... loss:59.57800215482712:   0%|                                                                                                | 1/5000 [02:16<189:52:07, 136.73s/it]

Val update: epoch: 0 |accuracy: 0.790862500667572 | f1: 0.8524629473686218 | auc: 0.8703110814094543 | treshold: 0.4
Test: accuracy: 0.7901341915130615 | f1: 0.8508327007293701 | auc: 0.8719138503074646 | 


train... loss:43.483878791332245:   0%|                                                                                               | 2/5000 [03:39<145:50:39, 105.05s/it]

Val update: epoch: 1 |accuracy: 0.8391621708869934 | f1: 0.8847926259040833 | auc: 0.9085134267807007 | treshold: 0.55
Test: accuracy: 0.8343126773834229 | f1: 0.8802516460418701 | auc: 0.9070823192596436 | 


...

train... loss:34.07372435927391:   0%|▏                                                                                               | 10/5000 [14:41<115:49:34, 83.56s/it]

Val update: epoch: 9 |accuracy: 0.8524887561798096 | f1: 0.8944133520126343 | auc: 0.9308153390884399 | treshold: 0.45
Test: accuracy: 0.848458468914032 | f1: 0.8904274702072144 | auc: 0.9278963804244995 | 


train... loss:33.853845089673996:   0%|▏                                                                                              | 11/5000 [16:04<115:33:15, 83.38s/it]

Val update: epoch: 10 |accuracy: 0.8641222715377808 | f1: 0.9003087878227234 | auc: 0.9330757856369019 | treshold: 0.46
Test: accuracy: 0.859412431716919 | f1: 0.8959183692932129 | auc: 0.9289408922195435 | 


train... loss:33.60004737973213:   0%|▏                                                                                               | 12/5000 [17:26<114:38:43, 82.74s/it]

In [ ]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items